In [1]:
import jsonlines
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from collections import Counter
import re

In [ ]:
'''
# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
'''

In [2]:
# ESTE ES EL SEGUNDO, QUE ES IDENTICO.


ruta_archivo = r'data\filtered_output\google-strict-filtered-reviews.jsonl' # RUTA DEL ARCHIVO QUE QUIEREN FILTRAR


# Conjunto de palabras vacías (stop words)
stopwords = set(stopwords.words('english'))


# Abrir el archivo JSONL
with jsonlines.open(ruta_archivo) as archivo_jsonl:
    # Recorrer cada línea en el archivo
    for linea in archivo_jsonl:
        if 'text' in linea and linea['text'] is not None:
            # Obtener el texto y rating de la línea
            texto = linea['text']
            rating = linea['rating']
            
            # Verificar si el rating es menor o igual a 2
            if rating <= 2:
                # Preprocesamiento del texto
                # Removal of invalid characters
                texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)

                # Tokenization
                tokens = nltk.word_tokenize(texto)

                # Remove stopwords
                
                tokens = [token for token in tokens if token not in stopwords]

                # Join tokens back into a single string
                preprocessed_text = ' '.join(tokens)

                # Tokenizar el texto preprocesado en oraciones y palabras
                oraciones = sent_tokenize(preprocessed_text)
                palabras = [word.lower() for oracion in oraciones for word in word_tokenize(oracion)]

                # Etiquetar las palabras con su tipo (POS tagging)
                palabras_etiquetadas = pos_tag(palabras)


Adjetivos:
good
bad
great
table
horrible
terrible
last
slow
poor
nice
first
cold
busy
ive
worst
much
small
wrong
new
many
long
disappointed
best
little
sure
cheese
old
better
expensive
next
ok
dry
dont
different
hot
hard
waitress
least
empty
worth
high
second
full
several
salad
original
right
awful
wait
happy
big
open
didnt
whole
special
im
fish
ready
extra
top
free
friendly
give
overall
entire
fresh
red
clean
rude
loud
large
restaurant
chicken
due
nasty
disappointing
worse
decent
cant
steak
half
front
olive
sick
average
warm
frozen
sad
raw
real
gross
dirty
tough
want
enough
super
black
short
fine
local
french
able
wasnt
less
available
ask
eat
ordered
regular
main
ridiculous
mexican
impressed
meal
young
multiple
rare
sweet
come
late
delicious
sat
favorite
low
unprofessional
total
huge
dish
hungry
outside
white
shrimp
wont
past
ill
attentive
soggy
single
sandwich
tiny
italian
server
lunch
excellent
saturday
drink
tried
simple
close
awesome
fried
cool
cheap
sorry
menu
sunday
live
u
mixed


#Imprimir palabras sin limitar la cantidad
for tipo, palabras in palabras_por_tipo.items():
    # Contar la frecuencia de las palabras
    contador_palabras = Counter(palabras)
    # Ordenar las palabras por frecuencia
    palabras_ordenadas = sorted(contador_palabras.items(), key=lambda x: x[1], reverse=True)
    print(f"{tipo}:")
    for palabra, frecuencia in palabras_ordenadas:
        print(f"{palabra}: {frecuencia}")
    print()

que en base al puntaje del bar, tome las reviews de 3 o menos, guarde palabras claves basadas en la lista de palabras qe esta separado en las 4 categorias y te devuelva las categorias con mas coincidencias.
esto podria hacerse con kmeans, con logical regresion, con vectorizacion igual a la del modelo del pi 1, o cualquier otra cosa.
pero si o si hay que hacer un filtro completo con las palabras correctas para el resultado esperado.
y eso si es un modelo de ml en teoria, pq sabemos que hay 4 categorias, pero no sabemos cual de esas van a ser las que van a resultar del analisis de las reviews. el problema que tengo con esto es que si son bares con pocas reviews y muchas reviews nulas, nos vale verga.
asi que como filtro inicial tenemos que filtrar las reviews que tengan text null, y a partir de ahi, volver a filtrar los gmap_id que tengan menos de x cantidad de reviews. 
de ahi podemos hacer el filtro inverso, agarrar el gmap_id de reviews y sacar todos los bares que no esten en los reviews.
eso nos dejaria muchos menos datos para trabajar en general.
y eso en teoria podemos correrlo como pipieline, guardar esos archivos filtrados, y subir esos como tablas en big query.

# FILTRO SUPER ESTRICTO DE REVIEWS

In [2]:
def preprocess_text(text):
    '''
    Function that cleans the text, using NLTK stopwords,
    removes abnormal characters, converts everything to lowercase, and returns the preprocessed text
    '''
    # Lowercasing
    text = text.lower()

    # Removal of invalid characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text



In [3]:

def load_jsonl_files(data_path):
    data = []
    for filename in os.listdir(data_path):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(data_path, filename)
            with jsonlines.open(file_path) as reader:
                for review in reader:
                    if 'text' in review and review['text']:
                        processed_text = preprocess_text(review['text'])
                        review['processed_text'] = processed_text
                        data.append(review)
    return data

# Usage:
data_path = 'data/data filtrada/Google Maps filtrado/Google Maps json/reviews-filtrados/'
data = load_jsonl_files(data_path)

In [4]:
from collections import Counter

# Extract the gmap_id from each object
gmap_ids = [obj['gmap_id'] for obj in data]

# Count the occurrences of each unique gmap_id
gmap_id_counts = Counter(gmap_ids)

# Filter the data based on the count condition
filtered_data = [obj for obj in data if gmap_id_counts[obj['gmap_id']] >= 50]

# Calculate the percentage of data lost
data_lost_percentage = ((len(data) - len(filtered_data)) / len(data)) * 100

# Print the filtered data count and percentage of data lost
print(f"Filtered data count: {len(filtered_data)}")
print(f"Percentage of data lost: {data_lost_percentage}%")

Filtered data count: 1116687
Percentage of data lost: 29.841492828569866%


In [5]:

# Extract the gmap_id and rating from each object
gmap_ids = [obj['gmap_id'] for obj in data]
ratings = [obj['rating'] for obj in data]

# Count the occurrences of each unique gmap_id
gmap_id_counts = Counter(gmap_ids)

# Filter the data based on the count condition
filtered_data = [obj for obj in data if gmap_id_counts[obj['gmap_id']] >= 50]

# Calculate the average rating for the original data
original_avg_rating = sum(ratings) / len(ratings)

# Calculate the average rating for the filtered data
filtered_ratings = [obj['rating'] for obj in filtered_data]
filtered_avg_rating = sum(filtered_ratings) / len(filtered_ratings)

# Calculate the percentage of data lost
data_lost_percentage = ((len(data) - len(filtered_data)) / len(data)) * 100

# Compare the average ratings
print(f"Original average rating: {original_avg_rating}")
print(f"Filtered average rating: {filtered_avg_rating}")
print(f"Data lost percentage: {data_lost_percentage}%")

Original average rating: 4.294831883382349
Filtered average rating: 4.277252264958758
Data lost percentage: 29.841492828569866%



# Extract the gmap_id from each object
gmap_ids = [obj['gmap_id'] for obj in data]

# Count the occurrences of each unique gmap_id
gmap_id_counts = Counter(gmap_ids)

# Sort the counts in descending order
sorted_counts = sorted(gmap_id_counts.items(), key=lambda x: x[1], reverse=True)

# Print the sorted counts
for gmap_id, count in sorted_counts:
    print(f"gmap_id: {gmap_id}, count: {count}")


# Extract the gmap_id from each object
gmap_ids = [obj['gmap_id'] for obj in data]

# Count the occurrences of each unique gmap_id
gmap_id_counts = Counter(gmap_ids)

# Filter the data based on the count condition
filtered_data = [obj for obj in data if gmap_id_counts[obj['gmap_id']] >= 50]

# Print the filtered data count
print(f"Filtered data count: {len(filtered_data)}")

# Define the output file path
output_folder = r'data\filtered_output'
output_filename = 'uber-strict-filtered-reviews.jsonl'
output_path = os.path.join(output_folder, output_filename)

# Save the filtered data to the output file
with jsonlines.open(output_path, mode='w') as writer:
    for review in filtered_data:
        writer.write(review)

# Print a success message
print(f"Filtered data saved to: {output_path}")

import json
import jsonlines
import pandas as pd

# Read the first file
file1_path = r'data\data filtrada\Google Maps filtrado\Google Maps json\google-strict-filtered-reviews.jsonl'
df1 = pd.read_json(file1_path, lines=True)

# Extract the unique 'gmap_id' values from the first file
gmap_ids = df1['gmap_id'].unique()

# Read the second file
file2_path = r'data\data filtrada\Google Maps filtrado\Google Maps json\filtered_bars.json'
filtered_data = []

# Save the filtered data as strict-filtered-bars.jsonl with UTF-8 encoding
output_path = r'data\strict-filtered-bars2.jsonl'
with open(output_path, 'w', encoding='utf-8') as file:
    for item in filtered_data:
        json.dump(item, file, ensure_ascii=False, check_circular=False)
        file.write('\n')

print("Filtering completed. Result saved as strict-filtered-bars.jsonl")

gmap_ids.size